This notebook walks through loading a pretrained word2vec model.     
The experiment was done on a large instance, since the model takes up about ~27GB in memory.     
We will extract the embedding mappings for words that occur in our training set, making it easier for the embeddings to fit in our limited colab instance.

### Downloading the pretrained word2vec model

In [2]:
!wget --header="Host: doc-0s-6o-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.56" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Cookie: AUTH_q396glqdcuo3cgp14o8502qdk4gju650_nonce=fp1i2rr60reqg" --header="Connection: keep-alive" "https://doc-0s-6o-docs.googleusercontent.com/docs/securesc/6fglfng6gsfg8svq9gbkvr4bflmoufjj/d22kn4f60d9r7jl8e7i6caasqq2i1g9h/1669733175000/17060459648485421300/17060459648485421300/1-3yuMwCo0HKDNqZOq2xcBmY-H-4cyele?e=download&ax=AEKYgyTCKLrdLYpYaj6XjfPygem-XBxSyU2Xz8pYuta6km3Xsu2mlye7KwwiYT-NQ-0EVQZnqHAkbsn3aBj4_jRGCANxY37M0ofKw6A52qXlff4-pa25NTUFQvsIRsbAMYalGZT5iC5STgtCgfRU0BBQqnuZL5AjDYWv7gpMonaiqcTXWSUCQ3BZ5-nt5H8iE_kjmBodOWaCgDNksrf4b7ZAN5ln0MKI5NtEe7oILibiV2A3YR8pi6ENV_ignsiQfV7He2AvUtw2BknKr1Y1Dgw98BY8o7SpOT1Tnzb4ikFkKAtXynjMonyPRc76i35W3Emka1eq1BWcaOe2QpglBm3hov5ESQrGobBItmXVzLmRNt_Ix_w37MzNIZeLlazF20oLMGuawJsi8apIw_h33e5VT4eKgNqE8_fg2d9IywCfKCHsvUzL9XLLNxWjHeRNhWDK8v-q8Rl_2iGCxlf0Dy5dWuH1_u66QIfPrl-I3xuBRVSrtp60XQMWd4Jj8baL_jfW-fjmKGmII_-SHD8IfjPlJEOdvfmJ0hHDPcTDxnG02OhpIpR_vn30sqfy29xtVehTOgnUQy_iWOKJI9h9-AXAJ77ealUv3E0rphpawtgcjMr_krG-7Wx1OD_mRZSW7O8nragMHLzSQjkhXaYvCsFq0Px-zL2f8c7R8wQzR-yN_slt9QjMrHx87Jf7UuU3Z3BPsBYc8dUy-kprlMux950fGLJ5cTz45g2GKncNEnoHAKsjuEeHa0LMOXnkju15QhS_RdN8uuZPDZ2NMLq6QlhfX1yH_SCctQhh42D8sM1yGH03RgeJT6ARrX37nn4kO87Gwis-__IIGdXuBvxqwZf1NR4_HO31PHqETqRVBHnyNQx-rUDj2RG-VJ-eXPBZdW_FABzLs6Fs-bxZ7IKQGbI7BoDYeC7tMT6jQ_F3q0dUFV-h9RsDApYI6FVBTzZIqJfIVz9ZVtPxNOqbtieNuU8407ejPqbOynUhBLuZmGo370nOq47ngxDY0U1FH6PI5UwNlZ74Rjc81ms9vzgoly3e-f_vRWz_EIbOzfvPEkKZ0UelvaAgaoY&uuid=4c293bfa-b1a8-4e33-a745-633fce558e5a&authuser=2&nonce=fp1i2rr60reqg&user=17060459648485421300&hash=vjprlkt80bnt8igemi6smtsvjmk6e37l" -c -O 'BioWordVec_PubMed_MIMICIII_d200.vec.bin'

--2022-11-29 14:47:49--  https://doc-0s-6o-docs.googleusercontent.com/docs/securesc/6fglfng6gsfg8svq9gbkvr4bflmoufjj/d22kn4f60d9r7jl8e7i6caasqq2i1g9h/1669733175000/17060459648485421300/17060459648485421300/1-3yuMwCo0HKDNqZOq2xcBmY-H-4cyele?e=download&ax=AEKYgyTCKLrdLYpYaj6XjfPygem-XBxSyU2Xz8pYuta6km3Xsu2mlye7KwwiYT-NQ-0EVQZnqHAkbsn3aBj4_jRGCANxY37M0ofKw6A52qXlff4-pa25NTUFQvsIRsbAMYalGZT5iC5STgtCgfRU0BBQqnuZL5AjDYWv7gpMonaiqcTXWSUCQ3BZ5-nt5H8iE_kjmBodOWaCgDNksrf4b7ZAN5ln0MKI5NtEe7oILibiV2A3YR8pi6ENV_ignsiQfV7He2AvUtw2BknKr1Y1Dgw98BY8o7SpOT1Tnzb4ikFkKAtXynjMonyPRc76i35W3Emka1eq1BWcaOe2QpglBm3hov5ESQrGobBItmXVzLmRNt_Ix_w37MzNIZeLlazF20oLMGuawJsi8apIw_h33e5VT4eKgNqE8_fg2d9IywCfKCHsvUzL9XLLNxWjHeRNhWDK8v-q8Rl_2iGCxlf0Dy5dWuH1_u66QIfPrl-I3xuBRVSrtp60XQMWd4Jj8baL_jfW-fjmKGmII_-SHD8IfjPlJEOdvfmJ0hHDPcTDxnG02OhpIpR_vn30sqfy29xtVehTOgnUQy_iWOKJI9h9-AXAJ77ealUv3E0rphpawtgcjMr_krG-7Wx1OD_mRZSW7O8nragMHLzSQjkhXaYvCsFq0Px-zL2f8c7R8wQzR-yN_slt9QjMrHx87Jf7UuU3Z3BPsBYc8dUy-kprlMux950fGLJ5cTz45g2GKncNEn

### Download the batch data

In [3]:
!wget --header="Host: northeastern-my.sharepoint.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.56" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://northeastern-my.sharepoint.com/personal/choudhari_p_northeastern_edu/_layouts/15/onedrive.aspx" --header="Cookie: MicrosoftApplicationsTelemetryDeviceId=48f04ca2-c4e6-0958-1183-50f51b84c217; MicrosoftApplicationsTelemetryFirstLaunchTime=1663628373112; ExcelWacDataCenter=PUS4; WacDataCenter=PUS4; PowerPointWacDataCenter=PUS4; SIMI=eyJzdCI6MH0=; rtFa=d/ticyP8j8Hk6o+X27HQ4F7ITVswmt12qS/hzeOshismQThFRUMyODEtQUFBMy00REFFLUFDOUItOUEzOThCOTIxNUU3IzEzMzEzMTg1NDU4NzUzMjk1MSM3MTkyNzlBMC0zMDlFLTIwMDAtRDQxMi0zQUNDMjAxQUZDMUUjQ0hPVURIQVJJLlAlNDBOT1JUSEVBU1RFUk4uRURVTSgneEtDD7sCQSG3gpdkZJYFci5yTYCtz7YVLHHd1HkukB/GqGkwgUNWzPi9UXIqDPbn0D1ogMq3mWNn+jCx1JVX78CJa0UyxkIdjauHLNMBk24IwrtVEdpGKRwbUxLARY7BL7PuCbCY7yRlOToZZ5NBIBQa7SnF86CdyxH4IPImz7VNV97JitlcC+6Tic+z3ySDGU1fFAB/fqMMKmiCY0ph9Zyh+J3xMdyolBaXlJFV70JS6M+Vn0LON3PE4UrO4BVZv9gjmtT5fe7M4iLABbGw1kqgCCPvy8i1dXAjkYobJ0FMibz33vMHk5ISrzBi8HIRblYEPhahwRx6g++7B5wAAAA=; FedAuth=77u/PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz48U1A+VjEyLDBoLmZ8bWVtYmVyc2hpcHwxMDAzMjAwMDdjNGMzMDQxQGxpdmUuY29tLDAjLmZ8bWVtYmVyc2hpcHxjaG91ZGhhcmkucEBub3J0aGVhc3Rlcm4uZWR1LDEzMzA4MzE4MzA4MDAwMDAwMCwxMzI4MzMwNTk4NTAwMDAwMDAsMTMzMTQyMTg4ODIxNDE4MDc4LDczLjIzNC4xMjIuMjUxLDY3LGE4ZWVjMjgxLWFhYTMtNGRhZS1hYzliLTlhMzk4YjkyMTVlNywsZDcxNDlhZDQtNDkyNC00NmJjLTgyOTMtMDU2MTkxOTlkMGY3LDcxOTI3OWEwLTMwOWUtMjAwMC1kNDEyLTNhY2MyMDFhZmMxZSwwMWQwN2JhMC00MDc1LTIwMDAtZjQzZS00ODc2NGYxZGI3ZDYsLDAsMTMzMTM3OTA0ODIxMjYyMTQ1LDEzMzE0MDQ2MDgyMTI2MjE0NSwsLGV5SjRiWE5mWTJNaU9pSmJYQ0pEVURGY0lsMGlMQ0o0YlhOZmMzTnRJam9pTVNJc0ltRmpjbk1pT2lKYlhDSjFjbTQ2ZFhObGNqcHlaV2RwYzNSbGNuTmxZM1Z5YVhSNWFXNW1iMXdpWFNJc0luQnlaV1psY25KbFpGOTFjMlZ5Ym1GdFpTSTZJbU5vYjNWa2FHRnlhUzV3UUc1dmNuUm9aV0Z6ZEdWeWJpNWxaSFVpTENKMWRHa2lPaUpPWW1SdVIwSldRVXByTWt0WE9GOXNOM1JKTlVGQkluMD0sMTMzMTMxODkwNTg3MjIwNDQwLDEzMzEzMTg1NDU4MDAwMDAwMCxjNjJhNTNhYy1iMGZkLTQyNDQtYmE3MC1kODQ1YmEwOTMwY2QsLCwsLCwwLCxHVXQ5UlpNYVhYVE9ENWZoK09yMDd6endwZWcyQXdZc3IxQ2JpeHkvc3BLam01OG5JZE12UW4xUFc2dlFWcjJjOUhYK2dnZGd3V0l3L1BuWWhnS25odms3UmQ0VU1DSVBZSnQ4b0VpUjNhL2hJWktTM3BJOHR3VFcwbzRqQThkTXZrOENSODU0Q0UwY0tha0YzRUNqSWJlMnMwMHRxWXdzVThxZjdxL2dLZVBySy9KKzEzV0xDVE9sREk5TmFiRXRCRHdDWU5ic2JYbU1wbFVwQUx5aVBwM2NDQURvTmxqSVB6NmdOSFpiOWg1WHdoMlFlcXhUdzlla3BvSGRJMHF6UDVwVnFCZ1JkKzF2dHdYc3VnS0RaUFNEbGRZWWtsZUZKcFBJd0FoUnFRazgxM2pMRlBTNHNtVWF0WDhNZWhCckNuVTcrVStXaHJXTk1nTG9nbkZkZUE9PTwvU1A+; odbn=1; WSS_FullScreenMode=false" --header="Connection: keep-alive" "https://northeastern-my.sharepoint.com/personal/choudhari_p_northeastern_edu/_layouts/15/download.aspx?UniqueId=e12465f9%2De777%2D449b%2D872d%2De2305eaed74f" -c -O 'training_batch_800_809.csv'

--2022-11-29 14:51:33--  https://northeastern-my.sharepoint.com/personal/choudhari_p_northeastern_edu/_layouts/15/download.aspx?UniqueId=e12465f9%2De777%2D449b%2D872d%2De2305eaed74f
Resolving northeastern-my.sharepoint.com (northeastern-my.sharepoint.com)... 52.107.251.59, 52.107.251.6, 52.107.251.2, ...
Connecting to northeastern-my.sharepoint.com (northeastern-my.sharepoint.com)|52.107.251.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 273646766 (261M) [application/octet-stream]
Saving to: ‘training_batch_800_809.csv’

training_batch_800_ 100%[===================>] 260.97M  4.03MB/s    in 44s     

2022-11-29 14:52:18 (5.95 MB/s) - ‘training_batch_800_809.csv’ saved [273646766/273646766]



### Reading the data

In [4]:
import pandas as pd
data = pd.read_csv('training_batch_800_809.csv')

Tokenize the AbstractText data.

In [10]:
from tensorflow.keras.preprocessing import text
imaginary_list = []

max_features = 283759
maxlen = 250

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data['AbstractText'].to_list())
train_sequence = tokenizer.texts_to_sequences(data['AbstractText'].to_list())

Extracting only the training words

In [11]:
training_words = tokenizer.word_index.keys()

In [13]:
len(training_words)

300765

### Loading the model in memory

In [14]:
from gensim.models import KeyedVectors
from gensim import models

word2vec_path = 'BioWordVec_PubMed_MIMICIII_d200.vec.bin'
w2v_model = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [19]:
universal_list = set(w2v_model.index_to_key)

In [20]:
remove_list = set(training_words).difference(universal_list)

In [21]:
len(remove_list)

42109

In [23]:
# Interesection
slice_vocab = set(training_words).intersection(universal_list)

In [24]:
len(slice_vocab)

258656

In [31]:
list(slice_vocab)[1]

'mesomeres'

### Embedding dictionary

In [35]:
# Dictionary mapping
embedding_lookup = {}
for i in list(slice_vocab):
    embedding_lookup[i] = w2v_model.__getitem__(i)

In [37]:
len(embedding_lookup)

258656

### Saving file as pickle and storing to S3 bucket

In [39]:
import pickle

In [40]:
with open('vector_subset.pickle', 'wb') as handle:
    pickle.dump(embedding_lookup, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [44]:
bucket='wherewillway'
key='vector_subset.pickle'

In [45]:
import logging
import boto3
from botocore.exceptions import ClientError
import os


def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [46]:
s3 = boto3.client('s3')
with open(key, "rb") as f:
    s3.upload_fileobj(f,bucket, key)